In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import os
import json
import tiktoken

In [54]:
def count_tokens(prompt):
    encoding = tiktoken.get_encoding("o200k_base")  # Use appropriate encoding for GPT model
    return len(encoding.encode(prompt))

train = pd.read_csv('/home/monsh/works/product-comments-qa/data/train_data.csv')
train['length'] = train['text'].apply(lambda x: len(x))
train.sort_values(by='length', inplace=True, ascending=False)
train = train[train['length'] > 60]

label = train.sample(n=20000)
label['tokens'] = label['text'].apply(count_tokens)

In [57]:
label.to_csv('/home/monsh/works/product-comments-qa/data/unannotate.csv', index=False)

In [19]:
import tiktoken
import openai

# from openai import OpenAI
# client = OpenAI()


In [ ]:
unannotate_df = pd.read_csv("/home/monsh/works/product-comments-qa/data/unannotate.csv")
model_name = "gpt-4o-mini"
batch_path = "/home/monsh/works/product-comments-qa/data/train_persona_user.jsonl"

prompt = \
"""
**Task:**  
You will play the role of a user who wants to download a new app from the Google Play Store. I will provide a **Thai comment** and the **domain of an example application**. Your task is to **rate the quality of the comment** based on the criteria below. The rating should be a whole number from **1 to 5**.

### **Rating Criteria**
**1 Star:**  
- The comment is mostly vulgar language.  
- The comment is unreadable (poor grammar, incomplete, or confusing).  

**2 Stars:**  
- The comment is a short, vague response with no useful information.  
- The comment is a long complaint but includes minor vulgar language.  

**3 Stars:**  
- The comment contains **one** of the following:  
  - Describes a feature of the app.  
  - Provides feedback or an idea to fix an issue.  

**4 Stars:**  
- The comment contains **two** of the following:  
  - Describes a feature of the app.  
  - Provides feedback or an idea to fix an issue.  
- Similar to a 5-star comment but with minor use of vulgar language.  

**5 Stars:**  
- The comment clearly explains a step-by-step idea or suggestion for improvement.  

**Instructions:**  
- Rate the quality of comments on a scale of 1-5, where 1 is the lowest quality and 5 is the highest quality.  
- No feedback is given on the rating.  
- Answer only with the numbers (1-5).  
- response in json from {1:'{answer1}',2:'{answer2}'}
"""

query_template = \
"""
Rate the following json comments:
`{comment}`
"""

if os.path.exists(batch_path):
    os.remove(batch_path)

with open(batch_path, "a+") as f:
    for i, comment in enumerate(tqdm(unannotate_df['text'])):
        data = {
            "custom_id": "sample_" + str(i), 
            "method": 
            "POST", "body": {
                "model": model_name, 
                "max_tokens": 50,
                "temperature": 0.4
            }, 
            "url": "/v1/chat/completions"}
        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": query_template.format(comment=comment)}
        ]
        data["body"]["messages"] = messages

        f.write(json.dumps(data, ensure_ascii=False) + "\n")

In [ ]:
from openai import OpenAI
client = OpenAI()

results = client.chat.completions.create(
    model=model_name,
    messages=messages,
)

batch_input_file = [
    client.files.create(
        file=open("/home/monsh/works/product-comments-qa/data/train_persona_user.jsonl", "rb"),
        purpose="batch"),
]

In [75]:
# batch_input_file_id = batch_input_file.id

batchs = [
    client.batches.create(
        input_file_id=batch.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
           "description": "Labeling user comments for app rating"
        }
    )
    for batch in batch_input_file
]

In [ ]:
files = [
    client.batches.retrieve(b.id).output_file_id for b in batchs
]

In [32]:
file_response = client.files.content(files[0])
llm_generate = file_response.text

In [33]:
with open("/home/monsh/works/product-comments-qa/data/train_persona_dev_sample_output.jsonl", "w") as f:
    f.write(llm_generate)

In [34]:
ai = pd.read_json(path_or_buf="/home/monsh/works/product-comments-qa/data/train_persona_dev_sample_output.jsonl", lines=True)
ai['response'] = ai['response'].apply(lambda x: int(x["body"]["choices"][0]["message"]["content"]))

ai_rating = ai['response'].values

In [38]:
for i, text in enumerate(train['text'][:100]):
    print(text)
    print("AI Rating:", ai_rating[i])
    print()

พบปัญหาที่เจอบ่อยมากในมือถือที่มีสเปกไม่สูงมาก เน็ตชอบแดงเวลาเล่นเกม กดตรวจสอบอินเตอร์เน็ต ก็ปกติดี ปรับกราฟฟิกต่ำสุด เพื่อลดการประมวลผลก็เหมือนเดิม แถมหนักกว่าเดิมด้วย หากจะมาบอกว่าลองเปลี่ยนไปใช้ wi-fi ขอบอกตรงนี้ว่าบางคนเขาก็ไม่มี! ด้วยทำการแก้ไข ไห้มีความเท่าเทียมทางอินเตอร์เน็ตด้วย อยากไห้ช่วยแค่นี้ แค่มาบอกปล่อยไว้ไห้คิดแล้วกันเพราะมันไม่ได้เจอแค่ คนกลุ่มเดียว
AI Rating: 4

ดีมากครับ แต่อยากให้มีโหมดออฟไลน์เล่นกับบอทและมีระดับความยากให้เลือกและสามารถใช้ได้ทุกฮีโร่ทุกสกินในเกม เล่นได้เฉพาะเวลาไม่มีเน็ต จะดีมากกกกกกเลยครับขอรองเลยอันนี้
AI Rating: 5

ตั้งเเต่อัพเดตใหม่มา เล่นมาหลายตาเเละมันจะชอบมีจังหวะเเลคเเล้วเเบบเเลคทั้งเกมเลยถ้าไม่เข้าเเล้วออกใหม่ก็ไม่หาย ทั้งๆที่เเต่ก่อนไม่เคยเป็นเล่นภาพสูงสุดตลอด พอปรับเป็นภาพตํ่ายังเเลคเหมือนเดิม งงนะเกม
AI Rating: 3

เกมดีครับเเต่ต้องปรับเรื่องการสุ่มผู้เล่นอยู่ในทีมเพราะบางคนอาจจะไม่ตั้งใจเล่นเเล้วอาจมีปัญหากับเพื่อนร่วมทีมครับขอบคุณครับ
AI Rating: 4

เกมมีความแหลก และค้างบ่อยมาก บางครั้งเด้งออก และกลายเป็นว่า AFK ควรมีระบบปิดเอฟเฟค ทั้งหม